**Выбор локации для скважины**

**Постановка задачи:**

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Содержание:**
    
    1. Загрузка и подготовка данных
        1.1. Загрузка библиотек
        1.2. Загрузка данных
        1.3. Первичное изучение массивов данных
        1.4. Удаление неинформативных признаков
        1.5. Создание обучающей и валидационной выборок
        1.6. Вывод по первому разделу
    2. Обучение и проверка ML моделей
        2.1. Формирование функции для обучения и проверки моделей
        2.2. Обучение и моделей и валидация для каждого региона
            2.2.1 Регион номер РАЗ
            2.2.2 Регион номер ДВА
            2.2.3 Регион номер ТРИ
        2.4. Вывод по второму разделу
    3. Подготовка к расчету прибыли
        3.1. Формирование минимального набора переменных для расчета прибыли
        3.2. Расчет минимального значения сырья для безубыточной разработки
        3.3. Сравнение полученного значения со средним значением запасов в регионе
        3.4. Вывод по разделу
     4. Расчет прибыли и рисков
         4.1. Подготовка функции для отбора самых перспективных скважин
         4.2. Расчет потенциальной прибыли с 200 лучших скважин региона
         4.3. Расчет рисков убытка при разработке
     5. Вывод по проекту

# Загрузка и подготовка данных

## Загрузка библиотек

In [1]:
# Для начала подгружаем необходимые библиотеки и инструменты

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# Отключим предупреждения в проекте

import warnings
warnings.filterwarnings('ignore')

## Загрузка данных

In [2]:
# Загружаем исходные массивы данных в отдельные переменные. Сначала для первого региона.

df_1 = pd.read_csv('geo_data_0.csv')

In [3]:
# Выводим его часть для визуального осмотра

df_1.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
# Теперь грузим второй датасет

df_2 = pd.read_csv('geo_data_1.csv')

In [5]:
# Смотрим на него

df_2.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [6]:
# Подгружаем третий набор данных

df_3 = pd.read_csv('geo_data_2.csv')

In [7]:
# Делаем внешнее ознакомление

df_3.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


## Первичное изучение массивов данных

In [8]:
# Для изучения наличия пропусков в данных и получения их количественной характеристики применяем медот info()

df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [9]:
# Аналогично поступаем для второго датасета

df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [10]:
# Ну и третий не обходим стороной

df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


**Выводы по первичному изучению данных**

Каждый из датасетов состоит из 100000 строк и 5 столбцов. Пропуски в данных отсутсвуют, что несколько облегчает их обработку. Каждый датасет содержит следующие столбцы:
1. id — уникальный идентификатор скважины:
    - формат данных Object корректировки не требует;
    - данный столбец является неинформативным с точки зрения машинного обучения, а потому его нужно будет отбросить;
2. f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы):
    - формат данных float не требует корректировки;
3. product — объём запасов в скважине (тыс. баррелей):
    - формат данных float не требует коррекции.

## Удаление неинформативных признаков

In [11]:
# Для машинного обучения признак, определяющий название скважины, не играет роли, 
# но усложняет работу модели. Поэтому от этих признаков можно без зазрения совести избавиться

df_1 = df_1.drop('id', axis=1)

In [12]:
# Проверяем результат

df_1.head()

,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647
3,-0.032172,0.139033,2.978566,168.620776
4,1.988431,0.155413,4.751769,154.036647


In [13]:
# То же делаем для второго датасета

df_2 = df_2.drop('id', axis=1)

In [14]:
df_2.head()

,f0,f1,f2,product
0,-15.001348,-8.276000,-0.005876,3.179103
1,14.272088,-3.475083,0.999183,26.953261
2,6.263187,-5.948386,5.001160,134.766305
3,-13.081196,-11.506057,4.999415,137.945408
4,12.702195,-8.147433,5.004363,134.766305


In [15]:
# И, конечно, не обходим стороной третий набор данных

df_3 = df_3.drop('id', axis=1)

In [16]:
# Разглядываем его

df_3.head()

,f0,f1,f2,product
0,-1.146987,0.963328,-0.828965,27.758673
1,0.262778,0.269839,-2.530187,56.069697
2,0.194587,0.289035,-5.586433,62.871910
3,2.236060,-0.553760,0.930038,114.572842
4,-0.515993,1.716266,5.899011,149.600746


## Создание обучающей и валидационной выборок

In [17]:
# Для того, чтобы иметь возможность обучить модель, а затем проверить ее знания на другом наборе данных,
# разделим полученные наборы признаков и целевых признаков первого региона на тестовую и валидационную выборки

df_train_1, df_valid_1 = train_test_split(df_1, random_state = 12345, test_size = 0.25)

In [18]:
# Аналогичные преобразования осуществляем с данными по второму региону

df_train_2, df_valid_2 = train_test_split(df_2, random_state = 12345, test_size = 0.25)

In [19]:
# Ну и о третьем регионе не забываем

df_train_3, df_valid_3 = train_test_split(df_3, random_state = 12345, test_size = 0.25)

## Отделение целевого признака от исходного набора

In [20]:
# Получим признаки тренировочного набора данных для первого региона

features_train_1 = df_train_1.drop('product', axis = 1) 

# Отделим целевой признак для первого региона

target_train_1 = df_train_1['product']

In [21]:
# То же для второго региона

features_train_2 = df_train_2.drop('product', axis = 1) 

# - || -

target_train_2 = df_train_2['product']

In [22]:
# И для третьего

features_train_3 = df_train_3.drop('product', axis = 1) 

### Готово!

target_train_3 = df_train_3['product']

In [23]:
### Получим признаки для валидационной выборки первого региона...

features_valid_1 = df_valid_1.drop('product', axis = 1) 

### Отделим целевой признак для первого региона

target_valid_1 = df_valid_1['product']

In [24]:
### То же для второго региона

features_valid_2 = df_valid_2.drop('product', axis = 1) 

### ...

target_valid_2 = df_valid_2['product']

In [25]:
### И для третьего

features_valid_3 = df_valid_3.drop('product', axis = 1) 

### Есть!

target_valid_3 = df_valid_3['product']

## Вывод по первому разделу

В первой части работы были осуществлены подготовительные действия для последующей обработки данных. 
В частности, сделано следующее:
    
    1. загружены необходимые библиотеки и инструменты;
    2. исходные датасеты записаны в переменные;
    3. произведен визуальный осмотр данных, дано описание их содержимого;
    4. произведено удаление неинформативного признака - названия скважины;
    5. датафреймы разбиты на тренировочную и валидационную выборки для каждого региона.
    6. целевой признак отделен от исходного набора данных.
Теперь можно приступать к обучению ML моделей.
 
</div>

# Обучение и проверка ML моделей

## Формирование функции для обучения и проверки моделей

По условию задачи, мы имеем право пользоваться только линейной регрессией. Что ж, приступим...

In [26]:
# Для уменьшения объемов кода на странице создадим функцию, которая сможет обучить модель линейной регрессии на тренировочной выборке,
# сделать предсказания на валидационной и выдать на-гора некоторые метрики 

def train_and_validate(data, features_train, target_train, features_valid, target_valid, number_of_reg):
    
    model = LinearRegression(fit_intercept = True, normalize = False)
    model.fit(features_train, target_train)
    
    predictions = pd.Series(model.predict(features_valid))
    
    mse = mean_squared_error(target_valid, predictions)
    r2 = r2_score(target_valid, predictions)
    
    print('Полученные значения метрик для {} региона:'.format(number_of_reg))
    print('Метрика MSE = {:.4f}'.format(mse))
    print('Метрика RMSE = {:.4f}'.format(mse ** 0.5))
    print('Метрика R2 = {:.4f}'.format(r2))
    print('Среднее значение предсказанного объема сырья = {:.4f}'.format(predictions.mean()))
    return model, mse, r2

## Обучение моделей и валидация для каждого региона

### Регион номер РАЗ

In [27]:
# Используем подготовленную функцию train_and_validate для того, чтобы обучить и в отдельную переменную записать 
# саму модель, сделать предсказания на валидационной выборке, а также вывести метрики MSE, RMSE, R2 
# и среднее количество предсказанного объема сырья в скважинах

model_1, mse_1, r2_1 = train_and_validate(df_1, features_train_1, target_train_1, features_valid_1, target_valid_1, 1)

Полученные значения метрик для 1 региона:
Метрика MSE = 1412.2129
Метрика RMSE = 37.5794
Метрика R2 = 0.2799
Среднее значение предсказанного объема сырья = 92.5926


In [28]:
# В исходный валидационный датасет добавляем столбец с предсказанными значениями объемов сырья

df_valid_1['predictions'] = model_1.predict(features_valid_1)

In [29]:
# И, конечно, визуальный осмотр результатов

df_valid_1.head()

,f0,f1,f2,product,predictions
71751,0.948970,-0.057547,2.095727,10.038645,95.894952
80493,0.992974,0.206671,-0.142278,114.551489,77.572583
2655,1.199854,-0.563356,-1.852991,132.603635,77.892640
53233,0.691422,-0.433347,0.564974,169.072125,90.175134
91141,0.420772,0.972638,0.736190,122.325180,70.510088


### Регион номер ДВА

In [30]:
# Повторяем совершенные действия для 2 региона

model_2, mse_2, r2_2 = train_and_validate(df_2, features_train_2, target_train_2, features_valid_2, target_valid_2, 2)

Полученные значения метрик для 2 региона:
Метрика MSE = 0.7976
Метрика RMSE = 0.8931
Метрика R2 = 0.9996
Среднее значение предсказанного объема сырья = 68.7285


In [31]:
# В исходный валидационный датасет добавляем столбец с предсказанными значениями объемов сырья

df_valid_2['predictions'] = model_2.predict(features_valid_2)

In [32]:
# Осмотрим результаты

df_valid_2.head()

,f0,f1,f2,product,predictions
71751,-0.371866,-1.862494,3.002210,80.859783,82.663314
80493,9.015122,-13.881455,1.995363,53.906522,54.431786
2655,-6.507568,-4.817448,1.003449,30.132364,29.748760
53233,14.560845,-10.667755,1.995175,53.906522,53.552133
91141,6.090476,-4.494723,0.013815,0.000000,1.243856


### Регион номер ТРИ

In [33]:
# Третий регоин хоть и в самом конце, но про него тоже не забываем

model_3, mse_3, r2_3 = train_and_validate(df_3, features_train_3, target_train_3, features_valid_3, target_valid_3, 3)

Полученные значения метрик для 3 региона:
Метрика MSE = 1602.3776
Метрика RMSE = 40.0297
Метрика R2 = 0.2052
Среднее значение предсказанного объема сырья = 94.9650


In [34]:
# В исходный валидационный датасет добавляем столбец с предсказанными значениями объемов сырья

df_valid_3['predictions'] = model_3.predict(features_valid_3)

In [35]:
# Осмотрим результаты

df_valid_3.head()

,f0,f1,f2,product,predictions
71751,-1.444717,-3.861599,2.225805,61.212375,93.599633
80493,-1.418617,1.276544,-0.976433,41.850118,75.105159
2655,-4.587649,-0.413199,1.649268,57.776581,90.066809
53233,1.871584,1.619101,4.273555,100.053761,105.162375
91141,-2.028785,4.128167,6.089547,109.897122,115.303310


## Вывод по второму разделу

На текущем этапе работы была обучена модель линейной регрессии для каждого из регионов, сделаны предсказания на валидационной выборке, получены метрики MSE, RMSE, R2 моделей и рассчитано среднее значение предсказанных объемов топлива.
    
Лучше всего модель обучилась на втором датасете - ее значени RMSE = 0,89 (против 37,6 у первой и 40 у третьей). Таким образом, вторая модель даст самые точные прогнозы по региону. 

При этом во втором регионе средние запасы сырья по скважинам самые скромные - 68,7 тыс. баррелей против 92,6 в первом и 94,9 в третьем. 

Двигаемся дальше.

# Подготовка к расчёту прибыли

## Формирование набора переменных для расчёта прибыли

In [36]:
# Количество исследуемых скважин в каждом регионе

RESEARCH_PLACES = 500

# Количество скважин для разработки в каждом регионе

DEVELOPMENT_PLACES = 200

# Бюджет на разработку скважин (тыс. руб.)

DEVELOPMENT_BUDGET = 10000000

# Доход с тысячи баррелей нефти (тыс. руб.)

REVENUE_PER_1000_BARREL = 450

## Расчет минимального значения сырья в скважине для безубыточной добычи

In [37]:
# Бюджет на одну скважину (тыс. руб.)

budget_per_place = DEVELOPMENT_BUDGET / DEVELOPMENT_PLACES
print('Бюджет на разработку одной скважины:', budget_per_place)

Бюджет на разработку одной скважины: 50000.0


In [38]:
# Рассчитаем, какое минимальное значение сырья должно быть добыто из скважины в среднем, чтобы добыча оказалась безубыточной

oil_min = DEVELOPMENT_BUDGET / (DEVELOPMENT_PLACES * REVENUE_PER_1000_BARREL)

print('Минимальное среднее значение добычи сырья: ', oil_min)

Минимальное среднее значение добычи сырья:  111.11111111111111


## Сравнение полученного значения со средним запасом сырья по регионам

In [39]:
# Для наглядности снова вызываем среднее количество предсказанных запасов сырья в 1 регионе

print('Средний запас сырья по первому региону: {} тыс. баррелей'.format(df_valid_1['predictions'].mean()))

Средний запас сырья по первому региону: 92.59256778438035 тыс. баррелей


In [40]:
# То же по 2 региону

print('Средний запас сырья по второму региону: {} тыс. баррелей'.format(df_valid_2['predictions'].mean()))

Средний запас сырья по второму региону: 68.72854689544602 тыс. баррелей


In [41]:
# И не забываем про 3 регион

print('Средний запас сырья по третьему региону: {} тыс. баррелей'.format(df_valid_3['predictions'].mean()))

Средний запас сырья по третьему региону: 94.96504596800489 тыс. баррелей


## Вывод по разделу

В данной главе проекта был рассчитан важный показатель - минимальное среднее количество добычи нефти для безубыточной разработки, который составил 111.1 тыс. баррелей сырья. 
    
Этот параметр оказался выше параметров средних запасов сырья в скважинах по каждому из регионов. Таким образом, нет очевидного понимания, что можно рассчитывать на доходы в какой-либо из областей добычи. 
    
Вот здесь-то и вступят в дело модели машинного обучения, с помощью которых можно сделать предположение о перспективности каждого из регионов и отобрать те скважины, которы должны принести доход.

# Расчёт прибыли и рисков 

## Подготовка функции для отбора самых перспективных скважин в регионе 

In [42]:
def profit_func(data, number_of_places):

    data_sorted = data.sort_values(by = 'predictions', ascending = False) # сортируем предсказания объемов по убыванию
    best_predictions = data_sorted['predictions'][:number_of_places] # отбираем в новую переменную первые number_of_points предсказанных значений
    best_places = data_sorted['product'][best_predictions.index] # отбираем реальные значения по индексам лучших предсказаний из переменной best_predictions
    best_places_sum = best_places.sum() # суммируем полученные значения
    profit = best_places_sum * REVENUE_PER_1000_BARREL - DEVELOPMENT_BUDGET # рассчитываем прибыль с разработки реальных месторождений
    
    return profit

## Расчет потенциальной прибыли с 200 лучших скважин каждого региона

In [43]:
print('Рассчетная прибыль с 200 лучших скважин первого региона: {:.2f} тыс. руб.'.format(profit_func(df_valid_1, 200)))

Рассчетная прибыль с 200 лучших скважин первого региона: 3320826.04 тыс. руб.


In [44]:
print('Рассчетная прибыль с 200 лучших скважин второго региона: {:.2f} тыс. руб.'.format(profit_func(df_valid_2, 200)))

Рассчетная прибыль с 200 лучших скважин второго региона: 2415086.70 тыс. руб.


In [45]:
print('Рассчетная прибыль с 200 лучших скважин третьего региона: {:.2f} тыс. руб.'.format(profit_func(df_valid_3, 200)))

Рассчетная прибыль с 200 лучших скважин третьего региона: 2710349.96 тыс. руб.


# Расчет рисков убытка при разработке

In [46]:
# Сформируем функцию для применения методики бутстрепа на 1000 выборок

state = np.random.RandomState(12345)

def bootstrap_1000(data):
    values = []
    lost = 0
    for i in range(1000):
        data_sample = data.sample(replace=True, random_state=state, n=500)
        profit = profit_func(data_sample, 200)
        
        if profit < 0:
            lost += 1
        values.append(profit)
       
    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    profit_mean = values.mean()
    risk = lost / 1000
    return profit_mean, lower, upper, risk

In [47]:
# Вызываем бутстреп для первого региона

mean_1, lower_1, upper_1, risk_1 = bootstrap_1000(df_valid_1)

print('Регион номер 1.')
print('Средняя прибыль по региону = {:.2f} млн.руб.'.format(mean_1/1000))
print('95% доверительный интервал: от {:.2f} до {:.2f} млн.руб.'.format(lower_1/1000, upper_1/1000))
print('Процент риска {:.2%}'.format(risk_1))

Регион номер 1.
Средняя прибыль по региону = 600.74 млн.руб.
95% доверительный интервал: от 12.95 до 1231.16 млн.руб.
Процент риска 2.00%


In [48]:
# Теперь для второго

mean_2, lower_2, upper_2, risk_2 = bootstrap_1000(df_valid_2)

print('Регион номер 2.')
print('Средняя прибыль по региону = {:.2f} млн.руб.'.format(mean_2/1000))
print('95% доверительный интервал: от {:.2f} до {:.2f} млн.руб.'.format(lower_2/1000, upper_2/1000))
print('Процент риска {:.2%}'.format(risk_2))

Регион номер 2.
Средняя прибыль по региону = 663.96 млн.руб.
95% доверительный интервал: от 206.48 до 1191.20 млн.руб.
Процент риска 0.10%


In [49]:
# И, конечно же, для третьего

mean_3, lower_3, upper_3, risk_3 = bootstrap_1000(df_valid_3)

print('Регион номер 3.')
print('Средняя прибыль по региону = {:.2f} млн.руб.'.format(mean_3/1000))
print('95% доверительный интервал: от {:.2f} до {:.2f} млн.руб.'.format(lower_3/1000, upper_3/1000))
print('Процент риска {:.2%}'.format(risk_3))

Регион номер 3.
Средняя прибыль по региону = 597.38 млн.руб.
95% доверительный интервал: от 1.73 до 1246.22 млн.руб.
Процент риска 2.50%


In [53]:
# Для наглядности представления результатов наших трудов оформим итоговую таблицу

final_table = pd.DataFrame({'Номер региона': ['Первый', 'Второй', 'Третий'],
        'RMSE модели': ['{:.2f}'.format(mse_1**0.5), '{:.2f}'.format(mse_2**0.5), '{:.2f}'.format(mse_3**0.5)],
        'Средняя прибыль, млн. руб.': ['{:.2f}'.format(mean_1/1000), '{:.2f}'.format(mean_2/1000), '{:.2f}'.format(mean_3/1000)], 
        'Доверительный интервал, млн. руб.': ['от {:.2f} до {:.2f}'.format(lower_1/1000, upper_1/1000), 
                                              'от {:.2f} до {:.2f}'.format(lower_2/1000, upper_2/1000),
                                              'от {:.2f} до {:.2f}'.format(lower_3/1000, upper_3/1000)],
       'Вероятность убытка': ['{:.2%}'.format(risk_1), '{:.2%}'.format(risk_2), '{:.2%}'.format(risk_3)]})

In [54]:
# Выводим таблицу на обозрение... барабанная дробь!

final_table

,Номер региона,RMSE модели,"Средняя прибыль, млн. руб.","Доверительный интервал, млн. руб.",Вероятность убытка
0,Первый,37.58,600.74,от 12.95 до 1231.16,2.00%
1,Второй,0.89,663.96,от 206.48 до 1191.20,0.10%
2,Третий,40.03,597.38,от 1.73 до 1246.22,2.50%


# Вывод по проекту

В данной работе были отточены навыки загрузки и подготовки данных к анализу, разбиения исходного датафрейма на тренировочный и валидационный, обучения модели линейной регрессии и определения ее метрик, расчета прогнозируемой прибыли, получения параметров доверительного интервала.
    
По итогам проекта стало ясно, что самым рискованным регионом с точки зрения возможных убытков является третий (вероятность убытка 2.5%, средняя прибыль 597.38 млн. руб.). Его мы отбрасываем автоматически, так как условия задачи требуют поиска риска меньше 2.5%. 
    
Далее по "ненадежности" идет первый регион (вероятность убытка 2.0%, средняя прибыль 600.74 млн. руб.). Вариант лучше третьего, но все же есть над чем работать!
    
Лучшим по всем показателям показал себя второй регион (вероятность убытка всего 0.1%, средняя прибыль самая большая - 663.96 млн. руб.). Таким образом, именно второй регион является предпочтительным для инвестирования денег в его разработку. 
</div>